# Part 1

In [27]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

__Using BeautifulSoup to get the table of postal codes__

In [28]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
soup = BeautifulSoup(result.content, 'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
df = df[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


__Removing Borough that has 'Not assigned' value__

In [29]:
df = df[df.Borough != 'Not assigned'].reset_index(drop = True)
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


__Assigning the same value in Borough for Neighbourhood with 'Not assigned' values__

In [30]:
for i in range(df.shape[0]):
    if df.iloc[i]['Neighbourhood'] == 'Not assigned':
        df.iloc[i]['Neighbourhood'] = df.iloc[i]['Borough']
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


__Grouping Neighbourhoods that have the same Postal Code__

In [31]:
new_df = pd.DataFrame(columns = {})
new_df = new_df.append(df.iloc[0])

for i in range(1, df.shape[0]):
    for j in range(new_df.shape[0]):
        if df.Postcode.iloc[i] == new_df.Postcode.iloc[j]:
            new_df.Neighbourhood.iloc[j] = new_df.Neighbourhood.iloc[j] + ', ' + df.Neighbourhood.iloc[i] 
            found = 1
        else:
            found = 0
    if found == 0:
        new_df = new_df.append(df.iloc[i])
          
new_df = new_df[['Postcode', 'Borough', 'Neighbourhood']]
new_df.rename(columns={'Postcode' : 'Postal Code'}, inplace = True)
new_df.head(10)  

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,"Rouge, Malvern"
9,M3B,North York,Don Mills North
10,M4B,East York,"Woodbine Gardens, Parkview Hill"
12,M5B,Downtown Toronto,"Ryerson, Garden District"


In [32]:
new_df.shape

(103, 3)

In [33]:
%store new_df

Stored 'new_df' (DataFrame)
